# Import Dependencies

In [206]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsRegressor

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# Data Acquisition

### Data Description

|Data|Description|
|----|-------------|
|bureau|All client's previous credits provided by other financial institutions that were reported to Credit Bureau.|
|bureau_balance|Monthly balances of previous credits in Credit Bureau.|
|POS_CASH_balance|Monthly balance snapshots of previous point of sales and cash loans that the applicant had with Home Credit.|
|credit_card_balance|Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.|
|previous_application|All previous applications for Home Credit loans of clients.|
|installments_payments|Repayment history for the previously disbursed credits in Home Credit.|

In [207]:
bureau = pd.read_csv(r"C:\Users\nene0\Documents\Datasets\home-credit-default-risk\bureau.csv")
bureau_bal = pd.read_csv(r"C:\Users\nene0\Documents\Datasets\home-credit-default-risk\bureau_balance.csv")
credit_bal = pd.read_csv(r"C:\Users\nene0\Documents\Datasets\home-credit-default-risk\credit_card_balance.csv")
installment = pd.read_csv(r"C:\Users\nene0\Documents\Datasets\home-credit-default-risk\installments_payments.csv")
cash_bal = pd.read_csv(r"C:\Users\nene0\Documents\Datasets\home-credit-default-risk\POS_CASH_balance.csv")

In [208]:
# For the sake of simplicity and to streamline this project, we'll be excluding the dataset containing information on previous loans.
# prev_app = pd.read_csv("C://Users//nene0//OneDrive//바탕 화면//Datasets//home-credit-default-risk////previous_application.csv")

# Data Analysis

## Bureau/Bureau Balance Dataset

In [209]:
bureau.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


In [210]:
bureau_bal.head()

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


In [211]:
bureau.shape, bureau_bal.shape

((1716428, 17), (27299925, 3))

In [212]:
bureau.isnull().sum()

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
CREDIT_CURRENCY                 0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        105553
DAYS_ENDDATE_FACT          633653
AMT_CREDIT_MAX_OVERDUE    1124488
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                 13
AMT_CREDIT_SUM_DEBT        257669
AMT_CREDIT_SUM_LIMIT       591780
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
AMT_ANNUITY               1226791
dtype: int64

In [213]:
bureau_bal.isnull().sum() #no missing values!

SK_ID_BUREAU      0
MONTHS_BALANCE    0
STATUS            0
dtype: int64

In [214]:
bureau.duplicated().sum()

0

In [215]:
merged_bureau = bureau.merge(bureau_bal, on='SK_ID_BUREAU', how='left')
merged_bureau.isnull().sum()

SK_ID_CURR                       0
SK_ID_BUREAU                     0
CREDIT_ACTIVE                    0
CREDIT_CURRENCY                  0
DAYS_CREDIT                      0
CREDIT_DAY_OVERDUE               0
DAYS_CREDIT_ENDDATE        1232569
DAYS_ENDDATE_FACT          5978831
AMT_CREDIT_MAX_OVERDUE    18130741
CNT_CREDIT_PROLONG               0
AMT_CREDIT_SUM                  13
AMT_CREDIT_SUM_DEBT        4234392
AMT_CREDIT_SUM_LIMIT      10671361
AMT_CREDIT_SUM_OVERDUE           0
CREDIT_TYPE                      0
DAYS_CREDIT_UPDATE               0
AMT_ANNUITY               10495530
MONTHS_BALANCE              942074
STATUS                      942074
dtype: int64

In [216]:
merged_bureau.shape

(25121815, 19)

In [217]:
merged_bureau

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE,STATUS
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.00,0.0,NaN,0.0,Consumer credit,-131,NaN,NaN,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.00,171342.0,NaN,0.0,Credit card,-20,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.50,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.00,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.00,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25121810,259355,5057750,Active,currency 1,-44,0,-30.0,NaN,0.0,0,11250.00,11250.0,0.0,0.0,Microloan,-19,NaN,NaN,NaN
25121811,100044,5057754,Closed,currency 1,-2648,0,-2433.0,-2493.0,5476.5,0,38130.84,0.0,0.0,0.0,Consumer credit,-2493,NaN,NaN,NaN
25121812,100044,5057762,Closed,currency 1,-1809,0,-1628.0,-970.0,NaN,0,15570.00,NaN,NaN,0.0,Consumer credit,-967,NaN,NaN,NaN
25121813,246829,5057770,Closed,currency 1,-1878,0,-1513.0,-1513.0,NaN,0,36000.00,0.0,0.0,0.0,Consumer credit,-1508,NaN,NaN,NaN


In [218]:
merged_bureau['CREDIT_ACTIVE'].value_counts()

CREDIT_ACTIVE
Closed      19101504
Active       5894501
Sold          125692
Bad debt         118
Name: count, dtype: int64

In [219]:
active_bur = merged_bureau[merged_bureau['CREDIT_ACTIVE'] != "Closed"]
active_bur.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE,STATUS
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,NaN,0.0,Credit card,-20,NaN,NaN,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN
5,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,NaN,NaN,NaN


In [220]:
active_bur.reset_index(drop=True, inplace=True)

In [221]:
active_bur.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE,STATUS
0,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.00,NaN,0.0,Credit card,-20,NaN,NaN,NaN
1,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN,NaN,NaN
2,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN,NaN,NaN
3,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN,NaN,NaN
4,215354,5714467,Active,currency 1,-273,0,27460.0,NaN,0.0,0,180000.0,71017.38,108982.62,0.0,Credit card,-31,NaN,NaN,NaN


In [222]:
active_bur[active_bur['DAYS_ENDDATE_FACT'].notnull()]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY,MONTHS_BALANCE,STATUS
187,161678,5715165,Sold,currency 1,-1319,0,-1134.0,-75.0,133308.81,0,202500.000,0.0,0.0,0.0,Consumer credit,-71,NaN,NaN,NaN
1817,310981,5716773,Active,currency 1,-403,0,-188.0,-403.0,0.00,0,70487.190,0.0,0.0,0.0,Consumer credit,-403,NaN,NaN,NaN
2563,351664,5717545,Sold,currency 1,-1911,0,-815.0,-134.0,42988.50,0,222750.000,NaN,NaN,0.0,Consumer credit,-22,NaN,NaN,NaN
5541,187117,5718814,Active,currency 1,-239,0,126.0,-56.0,NaN,0,110250.000,0.0,NaN,0.0,Consumer credit,-32,NaN,0.0,X
5542,187117,5718814,Active,currency 1,-239,0,126.0,-56.0,NaN,0,110250.000,0.0,NaN,0.0,Consumer credit,-32,NaN,-1.0,X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6016275,205991,5092696,Sold,currency 1,-1297,0,-201.0,-1050.0,0.00,0,337500.000,0.0,0.0,0.0,Another type of loan,-1050,NaN,NaN,NaN
6016647,450700,5066029,Sold,currency 1,-381,0,NaN,-111.0,0.00,0,45000.000,0.0,0.0,0.0,Credit card,-111,NaN,NaN,NaN
6016693,107312,5079711,Sold,currency 1,-2207,0,NaN,-79.0,6597.00,0,108000.000,0.0,0.0,0.0,Credit card,-79,NaN,NaN,NaN
6016697,137024,5079801,Sold,currency 1,-2161,0,-1858.0,-967.0,38516.67,0,109755.000,0.0,0.0,0.0,Consumer credit,-252,NaN,NaN,NaN


In [223]:
active_bur.drop(columns=['CREDIT_CURRENCY','DAYS_ENDDATE_FACT','AMT_CREDIT_MAX_OVERDUE','AMT_ANNUITY'], inplace=True)

In [224]:
active_bur.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6020311 entries, 0 to 6020310
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SK_ID_CURR              6020311 non-null  int64  
 1   SK_ID_BUREAU            6020311 non-null  int64  
 2   CREDIT_ACTIVE           6020311 non-null  object 
 3   DAYS_CREDIT             6020311 non-null  int64  
 4   CREDIT_DAY_OVERDUE      6020311 non-null  int64  
 5   DAYS_CREDIT_ENDDATE     5272083 non-null  float64
 6   CNT_CREDIT_PROLONG      6020311 non-null  int64  
 7   AMT_CREDIT_SUM          6020303 non-null  float64
 8   AMT_CREDIT_SUM_DEBT     5210827 non-null  float64
 9   AMT_CREDIT_SUM_LIMIT    4046133 non-null  float64
 10  AMT_CREDIT_SUM_OVERDUE  6020311 non-null  float64
 11  CREDIT_TYPE             6020311 non-null  object 
 12  DAYS_CREDIT_UPDATE      6020311 non-null  int64  
 13  MONTHS_BALANCE          5667888 non-null  float64
 14  ST

In [225]:
active_bur['MONTHS_BALANCE'].unique()

array([ nan,   0.,  -1.,  -2.,  -3.,  -4.,  -5.,  -6.,  -7.,  -8.,  -9.,
       -10., -11., -12., -13., -14., -15., -16., -17., -18., -19., -20.,
       -21., -22., -23., -24., -25., -26., -60., -61., -62., -63., -64.,
       -65., -66., -67., -68., -69., -70., -71., -72., -73., -74., -75.,
       -76., -77., -78., -79., -80., -81., -27., -28., -29., -52., -53.,
       -54., -37., -38., -39., -30., -31., -32., -33., -34., -35., -36.,
       -40., -41., -42., -43., -44., -45., -46., -47., -48., -49., -50.,
       -51., -55., -56., -57., -58., -59., -84., -85., -86., -87., -88.,
       -82., -83., -89., -90., -91., -92., -93., -94., -95., -96.])

In [226]:
active_bur['STATUS'].unique()

array([nan, 'C', '0', 'X', '1', '2', '5', '4', '3'], dtype=object)

In [227]:
active_bur['AMT_CREDIT_SUM'].skew()

30.769715461391396

In [228]:
active_bur['MONTHS_BALANCE'] = active_bur['MONTHS_BALANCE'].fillna(1)
active_bur['STATUS'] = active_bur['STATUS'].fillna('Unknown') #categorical data
active_bur['STATUS'] = active_bur['STATUS'].astype('object')
active_bur['AMT_CREDIT_SUM'] = merged_bureau['AMT_CREDIT_SUM'].fillna(merged_bureau['AMT_CREDIT_SUM'].median())

In [229]:
active_bur[active_bur['DAYS_CREDIT_ENDDATE'].isna()]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,MONTHS_BALANCE,STATUS
2,215354,5714465,Active,-203,0,NaN,0,464323.5,NaN,NaN,0.0,Credit card,-16,1.0,Unknown
10,238881,5714486,Active,-381,0,NaN,0,337500.0,520920.00,NaN,0.0,Consumer credit,-4,1.0,Unknown
12,238881,5714489,Active,-392,0,NaN,0,7033500.0,23679.00,228320.10,0.0,Credit card,-22,1.0,Unknown
20,435112,5714518,Active,-1483,0,NaN,0,252000.0,0.00,NaN,0.0,Credit card,-848,1.0,Unknown
29,303740,5714554,Active,-581,0,NaN,0,67500.0,263056.50,121690.17,0.0,Credit card,-22,1.0,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6019998,379272,5051674,Active,-638,0,NaN,0,58500.0,148.50,0.00,0.0,Credit card,-3,1.0,Unknown
6020274,254999,5055184,Active,-72,0,NaN,0,153310.5,75550.50,10398.15,0.0,Credit card,-20,1.0,Unknown
6020280,104893,5055269,Active,-51,0,NaN,0,153310.5,92882.25,10617.75,0.0,Credit card,-13,1.0,Unknown
6020287,169747,5055520,Active,-307,0,NaN,0,153310.5,0.00,112500.00,0.0,Credit card,-24,1.0,Unknown


In [230]:
active_bur[active_bur['DAYS_CREDIT_ENDDATE'].notnull()]

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,MONTHS_BALANCE,STATUS
0,215354,5714463,Active,-208,0,1075.0,0,91323.0,171342.00,NaN,0.0,Credit card,-20,1.0,Unknown
1,215354,5714464,Active,-203,0,528.0,0,225000.0,NaN,NaN,0.0,Consumer credit,-16,1.0,Unknown
3,215354,5714466,Active,-629,0,1197.0,0,90000.0,NaN,NaN,0.0,Consumer credit,-21,1.0,Unknown
4,215354,5714467,Active,-273,0,27460.0,0,2700000.0,71017.38,108982.62,0.0,Credit card,-31,1.0,Unknown
5,215354,5714468,Active,-43,0,79.0,0,180000.0,42103.80,0.00,0.0,Consumer credit,-22,1.0,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6020306,166279,5057599,Active,-99,0,997.0,0,153310.5,432787.50,0.00,0.0,Consumer credit,-7,1.0,Unknown
6020307,280228,5057642,Active,-575,0,813.0,0,153310.5,0.00,0.00,0.0,Credit card,-46,1.0,Unknown
6020308,288038,5057657,Active,-541,0,7.0,0,153310.5,10971.00,0.00,0.0,Consumer credit,-11,1.0,Unknown
6020309,433007,5057703,Active,-740,0,31128.0,0,153310.5,2466.00,NaN,0.0,Credit card,-433,1.0,Unknown


In [231]:
active_bur.isnull().sum()

SK_ID_CURR                      0
SK_ID_BUREAU                    0
CREDIT_ACTIVE                   0
DAYS_CREDIT                     0
CREDIT_DAY_OVERDUE              0
DAYS_CREDIT_ENDDATE        748228
CNT_CREDIT_PROLONG              0
AMT_CREDIT_SUM                  0
AMT_CREDIT_SUM_DEBT        809484
AMT_CREDIT_SUM_LIMIT      1974178
AMT_CREDIT_SUM_OVERDUE          0
CREDIT_TYPE                     0
DAYS_CREDIT_UPDATE              0
MONTHS_BALANCE                  0
STATUS                          0
dtype: int64

#### 1.) Freshest Balance
- Month of balance relative to application date (-1 means the freshest balance date)

In [232]:
balance_fresh = active_bur.groupby('SK_ID_CURR')[['MONTHS_BALANCE']].min().reset_index().rename(columns={"MONTHS_BALANCE":"BALANCE_FRESH"})
balance_fresh

,SK_ID_CURR,BALANCE_FRESH
0,100001,-18.0
1,100002,-34.0
2,100003,1.0
3,100005,-4.0
4,100008,1.0
...,...,...
252615,456247,-23.0
252616,456249,1.0
252617,456250,-27.0
252618,456253,-30.0


#### 2.) Latest Status

In [233]:
latest_status = active_bur.groupby('SK_ID_CURR')[['STATUS']].last().reset_index().rename(columns={"STATUS":"LAST_STATUS"})
latest_status

,SK_ID_CURR,LAST_STATUS
0,100001,0
1,100002,X
2,100003,Unknown
3,100005,0
4,100008,Unknown
...,...,...
252615,456247,0
252616,456249,Unknown
252617,456250,X
252618,456253,X


#### 3.) Reported Credits

In [234]:
reported_credits = active_bur.groupby('SK_ID_CURR')[['CREDIT_ACTIVE']].count().reset_index().rename(columns={"CREDIT_ACTIVE":"CREDIT_COUNT"})
reported_credits

,SK_ID_CURR,CREDIT_COUNT
0,100001,32
1,100002,20
2,100003,1
3,100005,8
4,100008,1
...,...,...
252615,456247,47
252616,456249,2
252617,456250,54
252618,456253,55


#### 4.) Recent/Mean Apply

In [235]:
recent_apply = active_bur.groupby('SK_ID_CURR')[['DAYS_CREDIT']].max().reset_index().rename(columns={"DAYS_CREDIT":"APPLY_CRED_MAX"})
recent_apply

,SK_ID_CURR,APPLY_CRED_MAX
0,100001,-49
1,100002,-103
2,100003,-606
3,100005,-62
4,100008,-78
...,...,...
252615,456247,-309
252616,456249,-483
252617,456250,-760
252618,456253,-713


In [236]:
long_apply = active_bur.groupby('SK_ID_CURR')[['DAYS_CREDIT']].min().reset_index().rename(columns={"DAYS_CREDIT":"APPLY_CRED_MIN"})
long_apply

,SK_ID_CURR,APPLY_CRED_MIN
0,100001,-559
1,100002,-1042
2,100003,-606
3,100005,-137
4,100008,-78
...,...,...
252615,456247,-712
252616,456249,-2612
252617,456250,-824
252618,456253,-919


#### 5.) Mean Credit Update

In [237]:
cred_update = active_bur.groupby('SK_ID_CURR')[['DAYS_CREDIT_UPDATE']].max().reset_index().rename(columns={"DAYS_CREDIT_UPDATE":"CRED_UPDATE"})
cred_update

,SK_ID_CURR,CRED_UPDATE
0,100001,-6
1,100002,-7
2,100003,-43
3,100005,-11
4,100008,-16
...,...,...
252615,456247,-12
252616,456249,-12
252617,456250,-23
252618,456253,-5


#### 6.) Credit Debt Sum

In [238]:
credit_debt = active_bur[active_bur['AMT_CREDIT_SUM_DEBT'].notnull()]
credit_debt.head()

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,MONTHS_BALANCE,STATUS
0,215354,5714463,Active,-208,0,1075.0,0,91323.00,171342.00,NaN,0.0,Credit card,-20,1.0,Unknown
4,215354,5714467,Active,-273,0,27460.0,0,2700000.00,71017.38,108982.62,0.0,Credit card,-31,1.0,Unknown
5,215354,5714468,Active,-43,0,79.0,0,180000.00,42103.80,0.00,0.0,Consumer credit,-22,1.0,Unknown
6,162297,5714471,Active,-1146,0,-484.0,0,42103.80,0.00,0.00,0.0,Credit card,-690,1.0,Unknown
7,162297,5714472,Active,-1146,0,-180.0,0,76878.45,0.00,0.00,0.0,Credit card,-690,1.0,Unknown


In [239]:
x=credit_debt[['DAYS_CREDIT', 'AMT_CREDIT_SUM', 'DAYS_CREDIT_UPDATE']]
y=credit_debt['AMT_CREDIT_SUM_DEBT']

cred_knn = KNeighborsRegressor(n_neighbors=5)
cred_knn.fit(x, y)

KNeighborsRegressor()

In [240]:
null_cred = active_bur[active_bur['AMT_CREDIT_SUM_DEBT'].isnull()]

In [241]:
cred_debt_pred = cred_knn.predict(null_cred[['DAYS_CREDIT', 'AMT_CREDIT_SUM', 'DAYS_CREDIT_UPDATE']])

In [242]:
def fill_missing_external(df, column_name, pred_list):
    missing_indices = df[df[column_name].isnull()].index

    for i, index in enumerate(missing_indices):
        if i < len(pred_list):
            df.at[index, column_name] = pred_list[i]
        else:
            break  # Break the loop if values_list is exhausted

In [243]:
fill_missing_external(active_bur, 'AMT_CREDIT_SUM_DEBT', cred_debt_pred)

In [244]:
active_bur.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6020311 entries, 0 to 6020310
Data columns (total 15 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   SK_ID_CURR              6020311 non-null  int64  
 1   SK_ID_BUREAU            6020311 non-null  int64  
 2   CREDIT_ACTIVE           6020311 non-null  object 
 3   DAYS_CREDIT             6020311 non-null  int64  
 4   CREDIT_DAY_OVERDUE      6020311 non-null  int64  
 5   DAYS_CREDIT_ENDDATE     5272083 non-null  float64
 6   CNT_CREDIT_PROLONG      6020311 non-null  int64  
 7   AMT_CREDIT_SUM          6020311 non-null  float64
 8   AMT_CREDIT_SUM_DEBT     6020311 non-null  float64
 9   AMT_CREDIT_SUM_LIMIT    4046133 non-null  float64
 10  AMT_CREDIT_SUM_OVERDUE  6020311 non-null  float64
 11  CREDIT_TYPE             6020311 non-null  object 
 12  DAYS_CREDIT_UPDATE      6020311 non-null  int64  
 13  MONTHS_BALANCE          6020311 non-null  float64
 14  ST

In [245]:
debt_med = active_bur.groupby('SK_ID_CURR')[['AMT_CREDIT_SUM_DEBT']].median().reset_index().rename(columns={"AMT_CREDIT_SUM_DEBT":"CRED_DEBT_MED"})
debt_med

,SK_ID_CURR,CRED_DEBT_MED
0,100001,113166.000
1,100002,245781.000
2,100003,0.000
3,100005,543087.000
4,100008,240057.000
...,...,...
252615,456247,2193390.000
252616,456249,81535.500
252617,456250,391731.615
252618,456253,171036.000


#### 6.) Overdue Credit Sum

In [246]:
ovr_cred = active_bur.groupby('SK_ID_CURR')[['AMT_CREDIT_SUM_OVERDUE']].sum().reset_index().rename(columns={"AMT_CREDIT_SUM_OVERDUE":"OVERDUE_CRED_SUM"})
ovr_cred

,SK_ID_CURR,OVERDUE_CRED_SUM
0,100001,0.0
1,100002,0.0
2,100003,0.0
3,100005,0.0
4,100008,0.0
...,...,...
252615,456247,0.0
252616,456249,0.0
252617,456250,0.0
252618,456253,0.0


#### 7.) Median Credit Sum

In [247]:
cred_sum_med = active_bur.groupby('SK_ID_CURR')[['AMT_CREDIT_SUM']].median().reset_index().rename(columns={"AMT_CREDIT_SUM":"CRED_SUM_MED"})
cred_sum_med

,SK_ID_CURR,CRED_SUM_MED
0,100001,81972.00
1,100002,41265.00
2,100003,45900.00
3,100005,22378.50
4,100008,80952.75
...,...,...
252615,456247,55795.50
252616,456249,157500.00
252617,456250,135000.00
252618,456253,0.00


### Merge Bureau Dataset

In [248]:
final_bureau = balance_fresh.merge(latest_status, on='SK_ID_CURR', how='left')
final_bureau = final_bureau.merge(reported_credits, on='SK_ID_CURR', how='left')
final_bureau = final_bureau.merge(recent_apply, on='SK_ID_CURR', how='left')
final_bureau = final_bureau.merge(long_apply, on='SK_ID_CURR', how='left')
final_bureau = final_bureau.merge(cred_update, on='SK_ID_CURR', how='left')
final_bureau = final_bureau.merge(debt_med, on='SK_ID_CURR', how='left')
final_bureau = final_bureau.merge(ovr_cred, on='SK_ID_CURR', how='left')
final_bureau = final_bureau.merge(cred_sum_med, on='SK_ID_CURR', how='left')
final_bureau

,SK_ID_CURR,BALANCE_FRESH,LAST_STATUS,CREDIT_COUNT,APPLY_CRED_MAX,APPLY_CRED_MIN,CRED_UPDATE,CRED_DEBT_MED,OVERDUE_CRED_SUM,CRED_SUM_MED
0,100001,-18.0,0,32,-49,-559,-6,113166.000,0.0,81972.00
1,100002,-34.0,X,20,-103,-1042,-7,245781.000,0.0,41265.00
2,100003,1.0,Unknown,1,-606,-606,-43,0.000,0.0,45900.00
3,100005,-4.0,0,8,-62,-137,-11,543087.000,0.0,22378.50
4,100008,1.0,Unknown,1,-78,-78,-16,240057.000,0.0,80952.75
...,...,...,...,...,...,...,...,...,...,...
252615,456247,-23.0,0,47,-309,-712,-12,2193390.000,0.0,55795.50
252616,456249,1.0,Unknown,2,-483,-2612,-12,81535.500,0.0,157500.00
252617,456250,-27.0,X,54,-760,-824,-23,391731.615,0.0,135000.00
252618,456253,-30.0,X,55,-713,-919,-5,171036.000,0.0,0.00


- BALANCE_FRESH: The freshest balance date relative to the application date.
- LAST_STATUS: The latest account status.
- CREDIT_COUNT: Number of credit accounts reported to the credit bureau.
- APPLY_CRED_MAX: The recent days before the current application did the client applied for credit bureau credit.
- APPLY_CRED_MIN: The oldest days before the current application did the client applied for credit bureau credit.
- CRED_UPDATE: The most recent days of credit information update.
- CRED_DEBT_MED: The median of the credit debt.
- OVERDUE_CRED_SUM: The sum of the credit overdue.
- CRED_SUM_MED: The median of the sum of the credit.

In [249]:
final_bureau.isnull().sum()

SK_ID_CURR          0
BALANCE_FRESH       0
LAST_STATUS         0
CREDIT_COUNT        0
APPLY_CRED_MAX      0
APPLY_CRED_MIN      0
CRED_UPDATE         0
CRED_DEBT_MED       0
OVERDUE_CRED_SUM    0
CRED_SUM_MED        0
dtype: int64

In [250]:
final_bureau.to_csv('cleaned_bureau.csv', index=False)

## Credit Card Balance
- Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.

In [251]:
credit_bal.isnull().sum() #Missing values are either 749816 or 305236 we will look at them separately.

SK_ID_PREV                         0
SK_ID_CURR                         0
MONTHS_BALANCE                     0
AMT_BALANCE                        0
AMT_CREDIT_LIMIT_ACTUAL            0
AMT_DRAWINGS_ATM_CURRENT      749816
AMT_DRAWINGS_CURRENT               0
AMT_DRAWINGS_OTHER_CURRENT    749816
AMT_DRAWINGS_POS_CURRENT      749816
AMT_INST_MIN_REGULARITY       305236
AMT_PAYMENT_CURRENT           767988
AMT_PAYMENT_TOTAL_CURRENT          0
AMT_RECEIVABLE_PRINCIPAL           0
AMT_RECIVABLE                      0
AMT_TOTAL_RECEIVABLE               0
CNT_DRAWINGS_ATM_CURRENT      749816
CNT_DRAWINGS_CURRENT               0
CNT_DRAWINGS_OTHER_CURRENT    749816
CNT_DRAWINGS_POS_CURRENT      749816
CNT_INSTALMENT_MATURE_CUM     305236
NAME_CONTRACT_STATUS               0
SK_DPD                             0
SK_DPD_DEF                         0
dtype: int64

In [252]:
credit_bal.head()

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,1800.0,1800.0,0.000,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,2250.0,2250.0,60175.080,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,2250.0,2250.0,26926.425,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,11925.0,11925.0,224949.285,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,27000.0,27000.0,443044.395,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


In [253]:
credit_bal.shape

(3840312, 23)

In [254]:
credit_bal[credit_bal['AMT_DRAWINGS_ATM_CURRENT'].isna()]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
45,2657726,399970,-5,0.0,225000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
47,1517613,121258,-6,0.0,135000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
49,2408643,104761,-4,0.0,225000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
52,1322825,215709,-5,0.0,180000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
60,1217908,162464,-5,0.0,45000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3840272,2463643,315621,-15,0.0,0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
3840303,1307188,385981,-9,0.0,0,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
3840306,1410474,255737,-13,0.0,180000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0
3840307,1036507,328243,-9,0.0,45000,NaN,0.0,NaN,NaN,0.0,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,0.0,Active,0,0


In [255]:
credit_bal[credit_bal['AMT_INST_MIN_REGULARITY'].isna()]

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,AMT_PAYMENT_CURRENT,AMT_PAYMENT_TOTAL_CURRENT,AMT_RECEIVABLE_PRINCIPAL,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
5362,2552584,208442,-52,0.0,0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,Active,0,0
5370,2681996,310976,-51,0.0,0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,Active,0,0
5382,1007356,432751,-52,0.0,0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,Active,0,0
5383,1721714,248980,-52,0.0,0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,Active,0,0
5385,1417366,247543,-54,0.0,0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,Active,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3838767,1321071,163997,-35,0.0,0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,Active,0,0
3838768,1455532,394176,-40,0.0,0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,Active,0,0
3838772,1578007,141528,-43,0.0,90000,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,Active,0,0
3838776,2051004,181711,-35,0.0,0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0,NaN,NaN,NaN,Active,0,0


In [256]:
credit_bal.fillna(0, inplace=True)

In [257]:
recent_cred_bal = credit_bal.groupby('SK_ID_CURR')[['MONTHS_BALANCE']].max().reset_index().rename(columns={"MONTHS_BALANCE":"CRED_FRESH"})
old_cred_bal = credit_bal.groupby('SK_ID_CURR')[['MONTHS_BALANCE']].min().reset_index().rename(columns={"MONTHS_BALANCE":"CRED_OLD"})
med_credit_limit = credit_bal.groupby('SK_ID_CURR')[['AMT_CREDIT_LIMIT_ACTUAL']].median().reset_index().rename(columns={"AMT_CREDIT_LIMIT_ACTUAL":"MED_CRED_LIMIT"})
med_amt_drawing = credit_bal.groupby('SK_ID_CURR')[['AMT_DRAWINGS_CURRENT']].median().reset_index().rename(columns={"AMT_DRAWINGS_CURRENT":"MED_CURRENT_DRAW"})
med_payment = credit_bal.groupby('SK_ID_CURR')[['AMT_PAYMENT_TOTAL_CURRENT']].median().reset_index().rename(columns={"AMT_PAYMENT_TOTAL_CURRENT":"MED_AMOUNT_PAID"})
med_principal = credit_bal.groupby('SK_ID_CURR')[['AMT_RECEIVABLE_PRINCIPAL']].median().reset_index().rename(columns={"AMT_RECEIVABLE_PRINCIPAL":"MED_AMOUNT_RECIEVE"})
mean_recieved = credit_bal.groupby('SK_ID_CURR')[['AMT_RECIVABLE']].median().reset_index().rename(columns={"AMT_RECIVABLE":"MED_AMOUNT_RECIEVED"})
mean_total = credit_bal.groupby('SK_ID_CURR')[['AMT_TOTAL_RECEIVABLE']].median().reset_index().rename(columns={"AMT_TOTAL_RECEIVABLE":"MED_TOTAL_RECIEVEABLE"})
max_num_drawing = credit_bal.groupby('SK_ID_CURR')[['CNT_DRAWINGS_CURRENT']].max().reset_index().rename(columns={"CNT_DRAWINGS_CURRENT":"MAX_DRAWINGS"})
min_inst = credit_bal.groupby('SK_ID_CURR')[['AMT_INST_MIN_REGULARITY']].max().reset_index().rename(columns={"AMT_INST_MIN_REGULARITY":"MED_MINMAX_INST"})

In [258]:
credit_final = recent_cred_bal.merge(old_cred_bal, on='SK_ID_CURR', how='left') 
credit_final = credit_final.merge(med_credit_limit, on='SK_ID_CURR', how='left')
credit_final = credit_final.merge(med_amt_drawing, on='SK_ID_CURR', how='left')
credit_final = credit_final.merge(med_payment, on='SK_ID_CURR', how='left')
credit_final = credit_final.merge(med_principal, on='SK_ID_CURR', how='left')
credit_final = credit_final.merge(mean_recieved, on='SK_ID_CURR', how='left')
credit_final = credit_final.merge(mean_total, on='SK_ID_CURR', how='left')
credit_final = credit_final.merge(max_num_drawing, on='SK_ID_CURR', how='left')
credit_final = credit_final.merge(min_inst, on='SK_ID_CURR', how='left')
credit_final

,SK_ID_CURR,CRED_FRESH,CRED_OLD,MED_CRED_LIMIT,MED_CURRENT_DRAW,MED_AMOUNT_PAID,MED_AMOUNT_RECIEVE,MED_AMOUNT_RECIEVED,MED_TOTAL_RECIEVEABLE,MAX_DRAWINGS,MED_MINMAX_INST
0,100006,-1,-6,270000.0,0.000,0.0000,0.0000,0.000,0.000,0,0.000
1,100011,-2,-75,180000.0,0.000,0.0000,0.0000,0.000,0.000,4,9000.000
2,100013,-1,-96,157500.0,0.000,0.0000,0.0000,0.000,0.000,7,7875.000
3,100021,-2,-18,675000.0,0.000,0.0000,0.0000,0.000,0.000,0,0.000
4,100023,-4,-11,135000.0,0.000,0.0000,0.0000,0.000,0.000,0,0.000
...,...,...,...,...,...,...,...,...,...,...,...
103553,456244,-1,-41,450000.0,0.000,154.2600,0.0000,0.000,0.000,8,23343.165
103554,456246,-2,-9,135000.0,526.275,9783.1800,752.0625,753.165,753.165,8,2250.000
103555,456247,-2,-96,180000.0,0.000,0.0000,0.0000,0.000,0.000,4,9000.000
103556,456248,-2,-24,900000.0,0.000,0.0000,0.0000,0.000,0.000,0,0.000


In [259]:
credit_final.to_csv('cleaned_credit.csv', index=False)

## Installment Payments
- Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
- There is a) one row for every payment that was made & b) one row each for missed payment.

In [260]:
installment.isnull().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
NUM_INSTALMENT_VERSION       0
NUM_INSTALMENT_NUMBER        0
DAYS_INSTALMENT              0
DAYS_ENTRY_PAYMENT        2905
AMT_INSTALMENT               0
AMT_PAYMENT               2905
dtype: int64

In [261]:
installment

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,NaN,67.500,NaN
13605397,1310347,414406,0.0,47,-1539.0,NaN,67.500,NaN
13605398,1308766,402199,0.0,43,-7.0,NaN,43737.435,NaN
13605399,1062206,409297,0.0,43,-1986.0,NaN,67.500,NaN


In [262]:
installment[installment['DAYS_ENTRY_PAYMENT'].isna()]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
3764207,1531600,103793,1.0,7,-668.0,NaN,49741.020,NaN
3764208,1947105,159974,1.0,24,-36.0,NaN,22849.515,NaN
3764209,1843773,167270,1.0,22,-20.0,NaN,48092.355,NaN
3764210,1691592,192536,1.0,5,-2561.0,NaN,7675.425,NaN
3764211,1531299,157088,0.0,11,-1847.0,NaN,67.500,NaN
...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,NaN,67.500,NaN
13605397,1310347,414406,0.0,47,-1539.0,NaN,67.500,NaN
13605398,1308766,402199,0.0,43,-7.0,NaN,43737.435,NaN
13605399,1062206,409297,0.0,43,-1986.0,NaN,67.500,NaN


In [263]:
installment[installment['DAYS_ENTRY_PAYMENT'].notnull()]

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
...,...,...,...,...,...,...,...,...
13605344,2006721,442291,1.0,3,-1311.0,-1318.0,2934.225,2934.225
13605345,1126000,428449,0.0,12,-301.0,-302.0,6793.470,6750.000
13605346,1519070,444122,1.0,5,-399.0,-407.0,4363.830,4363.830
13605347,2784672,444977,0.0,4,-157.0,-157.0,373.005,373.005


In [264]:
null_inst = installment[installment['DAYS_ENTRY_PAYMENT'].isna()]
notnull_inst = installment[installment['DAYS_ENTRY_PAYMENT'].notna()]

In [265]:
x=notnull_inst[['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER','DAYS_INSTALMENT','AMT_INSTALMENT']]
y=notnull_inst['DAYS_ENTRY_PAYMENT']

inst_knn = KNeighborsRegressor(n_neighbors=5)
inst_knn.fit(x,y)

KNeighborsRegressor()

In [266]:
entry_pred = inst_knn.predict(null_inst[['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER','DAYS_INSTALMENT','AMT_INSTALMENT']])

In [267]:
fill_missing_external(installment, 'DAYS_ENTRY_PAYMENT', entry_pred)

In [268]:
x=notnull_inst[['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER','DAYS_INSTALMENT','AMT_INSTALMENT']]
y=notnull_inst['AMT_PAYMENT']

pay_knn = KNeighborsRegressor(n_neighbors=5)
pay_knn.fit(x,y)

payment_pred = pay_knn.predict(null_inst[['NUM_INSTALMENT_VERSION','NUM_INSTALMENT_NUMBER','DAYS_INSTALMENT','AMT_INSTALMENT']])

In [269]:
fill_missing_external(installment, 'AMT_PAYMENT', payment_pred)

In [270]:
installment.isna().sum()

SK_ID_PREV                0
SK_ID_CURR                0
NUM_INSTALMENT_VERSION    0
NUM_INSTALMENT_NUMBER     0
DAYS_INSTALMENT           0
DAYS_ENTRY_PAYMENT        0
AMT_INSTALMENT            0
AMT_PAYMENT               0
dtype: int64

In [271]:
installment

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585
...,...,...,...,...,...,...,...,...
13605396,2186857,428057,0.0,66,-1624.0,-1320.0,67.500,66.123
13605397,1310347,414406,0.0,47,-1539.0,-1564.6,67.500,67.500
13605398,1308766,402199,0.0,43,-7.0,-26.4,43737.435,43734.303
13605399,1062206,409297,0.0,43,-1986.0,-2005.2,67.500,67.500


In [272]:
num_install = installment.groupby('SK_ID_CURR')[['SK_ID_PREV']].count().reset_index().rename(columns={"SK_ID_PREV":"COUNT_INSTALL"})
num_paid_install = installment.groupby('SK_ID_CURR')[['NUM_INSTALMENT_NUMBER']].median().reset_index().rename(columns={"NUM_INSTALMENT_NUMBER":"MED_PAID_INSTALL"})
pay_install = installment.groupby('SK_ID_CURR')[['DAYS_INSTALMENT']].min().reset_index().rename(columns={"DAYS_INSTALMENT":"INST_PAY_RECENT"})
paid_install = installment.groupby('SK_ID_CURR')[['DAYS_ENTRY_PAYMENT']].min().reset_index().rename(columns={"DAYS_ENTRY_PAYMENT":"INST_PAID"})
mean_amt_inst = installment.groupby('SK_ID_CURR')[['AMT_INSTALMENT']].median().reset_index().rename(columns={"AMT_INSTALMENT":"MED_INSTALL_AMT"})
mean_amt_paid = installment.groupby('SK_ID_CURR')[['AMT_PAYMENT']].median().reset_index().rename(columns={"AMT_PAYMENT":"MED_INSTALL_AMT_PAID"})

In [273]:
final_inst = num_install.merge(pay_install, on='SK_ID_CURR', how='left')
final_inst = final_inst.merge(num_paid_install, on='SK_ID_CURR', how='left')
final_inst = final_inst.merge(paid_install, on='SK_ID_CURR', how='left')
final_inst = final_inst.merge(mean_amt_inst, on='SK_ID_CURR', how='left')
final_inst = final_inst.merge(mean_amt_paid, on='SK_ID_CURR', how='left')
final_inst

,SK_ID_CURR,COUNT_INSTALL,INST_PAY_RECENT,MED_PAID_INSTALL,INST_PAID,MED_INSTALL_AMT,MED_INSTALL_AMT_PAID
0,100001,7,-2916.0,3.0,-2916.0,3980.925,3980.925
1,100002,19,-565.0,10.0,-587.0,9251.775,9251.775
2,100003,25,-2310.0,5.0,-2324.0,64275.615,64275.615
3,100004,3,-784.0,2.0,-795.0,5357.250,5357.250
4,100005,9,-706.0,5.0,-736.0,4813.200,4813.200
...,...,...,...,...,...,...,...
339582,456251,7,-210.0,4.0,-237.0,6605.910,6605.910
339583,456252,6,-2466.0,3.5,-2470.0,10074.465,10074.465
339584,456253,14,-2915.0,3.5,-2915.0,3973.095,3973.095
339585,456254,19,-291.0,5.0,-317.0,2296.440,2296.440


In [274]:
final_inst['INST_PAY_RECENT'] = final_inst['INST_PAY_RECENT'] * -1
final_inst['INST_PAID'] = final_inst['INST_PAID'] * -1

In [275]:
final_inst['DIFF_INST_PAID_DAYS'] = final_inst['INST_PAID'] - final_inst['INST_PAY_RECENT']
final_inst['DIFF_INST_PAID_AMT'] = final_inst['MED_INSTALL_AMT'] - final_inst['MED_INSTALL_AMT_PAID']

In [276]:
final_inst

,SK_ID_CURR,COUNT_INSTALL,INST_PAY_RECENT,MED_PAID_INSTALL,INST_PAID,MED_INSTALL_AMT,MED_INSTALL_AMT_PAID,DIFF_INST_PAID_DAYS,DIFF_INST_PAID_AMT
0,100001,7,2916.0,3.0,2916.0,3980.925,3980.925,0.0,0.0
1,100002,19,565.0,10.0,587.0,9251.775,9251.775,22.0,0.0
2,100003,25,2310.0,5.0,2324.0,64275.615,64275.615,14.0,0.0
3,100004,3,784.0,2.0,795.0,5357.250,5357.250,11.0,0.0
4,100005,9,706.0,5.0,736.0,4813.200,4813.200,30.0,0.0
...,...,...,...,...,...,...,...,...,...
339582,456251,7,210.0,4.0,237.0,6605.910,6605.910,27.0,0.0
339583,456252,6,2466.0,3.5,2470.0,10074.465,10074.465,4.0,0.0
339584,456253,14,2915.0,3.5,2915.0,3973.095,3973.095,0.0,0.0
339585,456254,19,291.0,5.0,317.0,2296.440,2296.440,26.0,0.0


In [277]:
final_inst.to_csv('cleaned_installment.csv', index=False)

## 4.) POS_CASH_balance
- Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.

In [278]:
cash_bal.isnull().sum()

SK_ID_PREV                   0
SK_ID_CURR                   0
MONTHS_BALANCE               0
CNT_INSTALMENT           26071
CNT_INSTALMENT_FUTURE    26087
NAME_CONTRACT_STATUS         0
SK_DPD                       0
SK_DPD_DEF                   0
dtype: int64

In [279]:
cash_bal.shape

(10001358, 8)

In [280]:
cash_bal.dropna(inplace=True)
cash_bal.reset_index(drop=True, inplace=True)

In [281]:
cash_bal

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0
...,...,...,...,...,...,...,...,...
9975169,2448283,226558,-20,6.0,0.0,Active,843,0
9975170,1717234,141565,-19,12.0,0.0,Active,602,0
9975171,1283126,315695,-21,10.0,0.0,Active,609,0
9975172,1082516,450255,-22,12.0,0.0,Active,614,0


In [282]:
num_loan = cash_bal.groupby('SK_ID_CURR')[['SK_ID_PREV']].count().reset_index().rename(columns={"SK_ID_PREV":"NUM_CASH_LOAN"})
recent_balance = cash_bal.groupby('SK_ID_CURR')[['MONTHS_BALANCE']].max().reset_index().rename(columns={"MONTHS_BALANCE":"FRESH_CASH_BAL"})
old_balance = cash_bal.groupby('SK_ID_CURR')[['MONTHS_BALANCE']].min().reset_index().rename(columns={"MONTHS_BALANCE":"OLD_CASH_BAL"})
avg_inst = cash_bal.groupby('SK_ID_CURR')[['CNT_INSTALMENT']].mean().reset_index().rename(columns={"CNT_INSTALMENT":"AVG_CASH_INST"})
avg_inst_left = cash_bal.groupby('SK_ID_CURR')[['CNT_INSTALMENT_FUTURE']].mean().reset_index().rename(columns={"CNT_INSTALMENT_FUTURE":"AVG_CASH_INST_LEFT"})

In [283]:
final_cash = num_loan.merge(recent_balance, on='SK_ID_CURR', how='left')
final_cash = final_cash.merge(old_balance, on='SK_ID_CURR', how='left')
final_cash = final_cash.merge(avg_inst, on='SK_ID_CURR', how='left')
final_cash = final_cash.merge(avg_inst_left, on='SK_ID_CURR', how='left')
final_cash

,SK_ID_CURR,NUM_CASH_LOAN,FRESH_CASH_BAL,OLD_CASH_BAL,AVG_CASH_INST,AVG_CASH_INST_LEFT
0,100001,9,-53,-96,4.000000,1.444444
1,100002,19,-1,-19,24.000000,15.000000
2,100003,28,-18,-77,10.107143,5.785714
3,100004,4,-24,-27,3.750000,2.250000
4,100005,10,-15,-24,11.700000,7.200000
...,...,...,...,...,...,...
337219,456251,8,-1,-8,7.875000,4.375000
337220,456252,7,-76,-82,6.000000,3.000000
337221,456253,17,-57,-96,6.705882,2.000000
337222,456254,20,-1,-11,14.900000,10.350000


In [284]:
final_cash['AVG_CASH_INST'] = round(final_cash['AVG_CASH_INST'])
final_cash['AVG_CASH_INST_LEFT'] = round(final_cash['AVG_CASH_INST_LEFT'])

In [285]:
final_cash['AVG_CASH_INST_PAID'] = final_cash['AVG_CASH_INST'] - final_cash['AVG_CASH_INST_LEFT']

In [286]:
final_cash

,SK_ID_CURR,NUM_CASH_LOAN,FRESH_CASH_BAL,OLD_CASH_BAL,AVG_CASH_INST,AVG_CASH_INST_LEFT,AVG_CASH_INST_PAID
0,100001,9,-53,-96,4.0,1.0,3.0
1,100002,19,-1,-19,24.0,15.0,9.0
2,100003,28,-18,-77,10.0,6.0,4.0
3,100004,4,-24,-27,4.0,2.0,2.0
4,100005,10,-15,-24,12.0,7.0,5.0
...,...,...,...,...,...,...,...
337219,456251,8,-1,-8,8.0,4.0,4.0
337220,456252,7,-76,-82,6.0,3.0,3.0
337221,456253,17,-57,-96,7.0,2.0,5.0
337222,456254,20,-1,-11,15.0,10.0,5.0


In [287]:
final_cash.to_csv("cleaned_POS.csv", index=False)